<a href="https://colab.research.google.com/github/sanketh96/Chatbot-2.0/blob/master/NER_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import copy

In [0]:
url = 'https://raw.githubusercontent.com/sanketh96/NER/master/combined_data.csv?token=AOeG7pC_lfTmYmLqsms8t8sc3cPNTlzJks5clKtrwA%3D%3D'
data = pd.read_csv(url)

In [0]:
data.tail()

,Unnamed: 0,POS,sentence_id,tag1,tag2,tag3,tag4,word,final_tag
220918,86629,TO,8733,B-PP,o,o,o,to,o
220919,86630,DT,8733,B-NP,o,o,o,this,o
220920,86631,JJ,8733,I-NP,o,o,o,high,o
220921,86632,NN,8733,I-NP,o,o,o,office,o
220922,86633,.,8733,o,o,o,o,.,o


In [0]:
data.drop(columns=['Unnamed: 0'])

,POS,sentence_id,tag1,tag2,tag3,tag4,word,final_tag
0,NNP,0,B-NP,o,o,o,URL,o
1,:,0,NP,o,o,o,;,o
2,NN,0,B-NP,B-URLS,o,o,http,o
3,:,0,NP,I-URLS,o,o,:,o
4,NN,0,B-NP,I-URLS,o,o,/,o
5,NN,0,I-NP,I-URLS,o,o,/,o
6,JJ,0,I-NP,I-URLS,o,o,www.indiapost.gov.in,o
7,NN,0,I-NP,I-URLS,o,o,/,o
8,NNP,0,I-NP,I-URLS,o,o,NSC.aspx,o
9,NNP,0,I-NP,o,o,o,Time,o


In [0]:
data['final_tag'].value_counts()

o        193632
B-PER      4614
B-LOC      3524
I-PER      3375
B-DT       2842
B-NUM      2834
I-ORG      2094
B-ORG      1662
B-GPE      1360
I-LOC      1339
I-NUM      1043
I-DT        825
I-GPE       767
I-EVE       458
B-EVE       326
B-ART       141
I-ART        87
Name: final_tag, dtype: int64

In [0]:
len(data['final_tag'].isna()) == len(data) # no missing tags

True

In [0]:
len(data['word'].isna()) == len(data) # no missing words

True

In [0]:
words = list(data['word'].values)

In [0]:
len(words)

220923

In [0]:
[word for word in words if not(word.isalnum())]

AttributeError: ignored

In [0]:
len([word for word in words if str(word).isalnum()])

186375

In [0]:
n_words = len(words)

In [0]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["final_tag"].values.tolist())]
        self.grouped = self.data.groupby("sentence_id").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped[self.n_sent]
            self.n_sent += 1
            return s
        except:
            return None


In [0]:
getter = SentenceGetter(data)

In [0]:
getter.sentences

[[('URL', 'NNP', 'o'),
  (';', ':', 'o'),
  ('http', 'NN', 'o'),
  (':', ':', 'o'),
  ('/', 'NN', 'o'),
  ('/', 'NN', 'o'),
  ('www.indiapost.gov.in', 'JJ', 'o'),
  ('/', 'NN', 'o'),
  ('NSC.aspx', 'NNP', 'o'),
  ('Time', 'NNP', 'o'),
  (':', ':', 'o'),
  ('3', 'CD', 'o'),
  ('.', '.', 'o')],
 [('30', 'CD', 'o'),
  ('P.M', 'NNP', 'o'),
  ('National', 'NNP', 'o'),
  ('Savings', 'NNP', 'o'),
  ('Certificates', 'NNS', 'o'),
  ('(', '(', 'o'),
  ('NSC', 'NNP', 'o'),
  (')', 'SYM', 'o')],
 [('NSC', 'NNP', 'o'), ('VIII', 'NNP', 'o'), ('Issue', 'NN', 'o')],
 [('*', 'SYM', 'o'),
  ('Scheme', 'NNP', 'o'),
  ('specially', 'RB', 'o'),
  ('designed', 'VBD', 'o'),
  ('for', 'IN', 'o'),
  ('Government', 'NNP', 'B-PER'),
  ('employees', 'NNS', 'I-PER'),
  (',', ',', 'o'),
  ('Businessmen', 'NNS', 'o'),
  ('and', 'CC', 'o'),
  ('other', 'JJ', 'o'),
  ('salaried', 'JJ', 'o'),
  ('classes', 'NNS', 'o'),
  ('who', 'WP', 'o'),
  ('are', 'VBP', 'o'),
  ('Income', 'NNP', 'o'),
  ('Tax', 'NNP', 'o'),
  ('ass

In [0]:
getter.get_next()

[('30', 'CD', 'o'),
 ('P.M', 'NNP', 'o'),
 ('National', 'NNP', 'o'),
 ('Savings', 'NNP', 'o'),
 ('Certificates', 'NNS', 'o'),
 ('(', '(', 'o'),
 ('NSC', 'NNP', 'o'),
 (')', 'SYM', 'o')]

In [0]:
sentences = getter.sentences

In [0]:
cleaned_sentences = copy.deepcopy(sentences)
for outer_index in range(len(sentences)):
  for inner_index in range(len(sentences[outer_index])):
    if type(sentences[outer_index][inner_index][0]) is not str:
      print(sentences[outer_index])
      print(sentences[outer_index][inner_index])
      temp_list = list(sentences[outer_index][inner_index])
      temp_list[0] = ''
      cleaned_sentences[outer_index][inner_index] = tuple(temp_list)
      print("After cleaning : ")
      print(cleaned_sentences[outer_index])
      print(cleaned_sentences[outer_index][inner_index])

[('39', 'CD', 'o'), ('in', 'IN', 'o'), (')', 'SYM', 'o'), ('per', 'IN', 'o'), ('hour', 'NN', 'o'), ('[', '(', 'o'), ('95', 'CD', 'o'), (']', ')', 'o'), ('[', '(', 'o'), ('96', 'CD', 'o'), (']', ')', 'o'), ('*', 'SYM', 'o'), ('Heavy', 'NNP', 'o'), ('rain', 'NN', 'o'), ('â\x80\x94', 'NN', 'o'), ('when', 'WRB', 'o'), ('the', 'DT', 'o'), ('precipitation', 'NN', 'o'), ('rate', 'NN', 'o'), ('is', 'VBZ', 'o'), (nan, 'SYM', 'o'), ('7', 'CD', 'o'), ('.', '.', 'o')]
(nan, 'SYM', 'o')
After cleaning : 
[('39', 'CD', 'o'), ('in', 'IN', 'o'), (')', 'SYM', 'o'), ('per', 'IN', 'o'), ('hour', 'NN', 'o'), ('[', '(', 'o'), ('95', 'CD', 'o'), (']', ')', 'o'), ('[', '(', 'o'), ('96', 'CD', 'o'), (']', ')', 'o'), ('*', 'SYM', 'o'), ('Heavy', 'NNP', 'o'), ('rain', 'NN', 'o'), ('â\x80\x94', 'NN', 'o'), ('when', 'WRB', 'o'), ('the', 'DT', 'o'), ('precipitation', 'NN', 'o'), ('rate', 'NN', 'o'), ('is', 'VBZ', 'o'), ('', 'SYM', 'o'), ('7', 'CD', 'o'), ('.', '.', 'o')]
('', 'SYM', 'o')
[('0', 'CD', 'o'), ('in', 

In [0]:
for outer_index in range(len(sentences)):
  for inner_index in range(len(sentences[outer_index])):
    if type(sentences[outer_index][inner_index][1]) is not str:
      print(sentences[outer_index])
      print(sentences[outer_index][inner_index])
      temp_list = list(sentences[outer_index][inner_index])
      temp_list[1] = ''
      cleaned_sentences[outer_index][inner_index] = tuple(temp_list)
      print("After cleaning : ")
      print(cleaned_sentences[outer_index])
      print(cleaned_sentences[outer_index][inner_index])

[('*', 'SYM', 'o'), ('Source', 'NN', 'o'), (':', ':', 'o'), ('mitbbs.com', 'NNP', 'B-ORG'), ('BBS', 'NNP', 'I-ORG'), (',', ',', 'o'), ('http:', nan, 'o'), ('-LSB-', 'JJ', 'o'), ('FROM', 'IN', 'o'), (':', ':', 'o'), ('69.15.', 'CD', 'B-NUM'), ('-RSB-', 'JJ', 'o')]
('http:', nan, 'o')
After cleaning : 
[('*', 'SYM', 'o'), ('Source', 'NN', 'o'), (':', ':', 'o'), ('mitbbs.com', 'NNP', 'B-ORG'), ('BBS', 'NNP', 'I-ORG'), (',', ',', 'o'), ('http:', '', 'o'), ('-LSB-', 'JJ', 'o'), ('FROM', 'IN', 'o'), (':', ':', 'o'), ('69.15.', 'CD', 'B-NUM'), ('-RSB-', 'JJ', 'o')]
('http:', '', 'o')
[('It', 'PRP', 'o'), ('is', 'VBZ', 'o'), ('also', 'RB', 'o'), ('known', 'VBN', 'o'), ('as', 'IN', 'o'), ('the', 'DT', 'o'), ('bachelor', 'NN', 'o'), ('of', 'IN', 'o'), ('veterinary', 'JJ', 'o'), ('medicine', 'NN', 'o'), ('and', 'CC', 'o'), ('surgery', 'NN', 'o'), ('at', 'IN', 'o'), ('some', 'DT', 'o'), ('universities', 'NNS', 'o'), (';', ':', 'o'), ('BVSc', 'NNP', 'o'), (',', ',', 'o'), ('BVMS', nan, 'o'), (',', 

In [0]:
cleaned_words = []
for sent in cleaned_sentences:
  for word in sent:
    cleaned_words.append(word[0])

In [0]:
tags = list(set(data["final_tag"].values))
n_tags = len(tags); n_tags


17

In [0]:
max_len = 50
word2idx = {w: i for i, w in enumerate(cleaned_words)}
tag2idx = {t: i for i, t in enumerate(tags)}

In [0]:
tag2idx

{'B-ART': 12,
 'B-DT': 6,
 'B-EVE': 9,
 'B-GPE': 5,
 'B-LOC': 3,
 'B-NUM': 13,
 'B-ORG': 4,
 'B-PER': 8,
 'I-ART': 1,
 'I-DT': 7,
 'I-EVE': 11,
 'I-GPE': 16,
 'I-LOC': 2,
 'I-NUM': 14,
 'I-ORG': 0,
 'I-PER': 15,
 'o': 10}

In [0]:
from keras.preprocessing.sequence import pad_sequences
X = [[word2idx[w[0]] for w in s] for s in cleaned_sentences]

Using TensorFlow backend.


In [0]:
X[0]

[214142,
 218494,
 214144,
 220868,
 219255,
 219255,
 134295,
 219255,
 134297,
 214153,
 220868,
 215606,
 220922]

In [0]:
X = pad_sequences(maxlen=max_len, sequences=X, padding="post", value=n_words - 1)


In [0]:
y = [[tag2idx[w[2]] for w in s] for s in cleaned_sentences]


In [0]:
for tag in y:
  print(tag)

[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10]
[10, 10, 10, 10, 10, 10, 10, 10]
[10, 10, 10]
[10, 10, 10, 10, 10, 8, 15, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 13, 14, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10]
[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 13, 10, 10, 10, 10, 10, 10, 10, 10]
[10, 10]
[10, 10, 10, 13, 14, 10]
[10, 10, 13, 10, 10]
[10, 10, 10]
[10, 10, 10, 10, 10, 10, 10, 13, 14, 10, 10, 10, 10, 13, 14, 10]
[10, 10, 13, 10, 10, 10, 13, 14, 10, 10, 10, 10, 10, 10, 10, 10, 10, 13, 14, 10, 10, 10, 13, 10, 10, 10, 13, 10, 10, 10, 13, 10, 10, 10, 13, 14, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10]
[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 13, 10, 10, 10, 10, 10, 10, 10, 10]
[10, 10]
[6, 10, 10, 13, 14, 10]
[10,

In [0]:
y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=tag2idx["o"])

In [0]:
count = 0
for tag in y:
  if np.count_nonzero(tag == tag2idx["o"]) == len(tag):
    count += 1
print(count)

2945


In [0]:
len(y)

8564

In [0]:
# So about 3k out of 8500+ sentences are just filled with o's, in other words they aren't that useful(?)..

In [0]:
from keras.utils import to_categorical
y = [to_categorical(i, num_classes=n_tags) for i in y]

In [0]:
from sklearn.model_selection import train_test_split


In [0]:
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.1)


In [0]:
X_tr

array([[204105, 217144, 220782, ..., 109666, 109667, 220922],
       [220526, 220376, 220709, ..., 220922, 220922, 220922],
       [220850, 201098, 220890, ..., 220922, 220922, 220922],
       ...,
       [220078, 220347, 220403, ..., 220922, 220922, 220922],
       [190856, 220780, 210404, ..., 219609, 147365, 147366],
       [197869, 185225, 220709, ..., 220922, 220922, 220922]], dtype=int32)

In [0]:
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional

In [0]:
input = Input(shape=(max_len,))
model = Embedding(input_dim=n_words, output_dim=50, input_length=max_len)(input)
model = Dropout(0.1)(model)
model = Bidirectional(LSTM(units=100, return_sequences=True, recurrent_dropout=0.1))(model)
out = TimeDistributed(Dense(n_tags, activation="softmax"))(model)  # softmax output layer


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
model = Model(input, out)


In [0]:
import keras.backend as K

def f1_score(y_true, y_pred):

    # Count positive samples.
    c1 = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    c2 = K.sum(K.round(K.clip(y_pred, 0, 1)))
    c3 = K.sum(K.round(K.clip(y_true, 0, 1)))

    # If there are no true samples, fix the F1 score at 0.
    if c3 == 0:
        return 0

    # How many selected items are relevant?
    precision = c1 / c2

    # How many relevant items are selected?
    recall = c1 / c3

    # Calculate f1_score
    f1_score = 2 * (precision * recall) / (precision + recall)
    return f1_score


In [0]:
model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy", f1_score])


In [0]:
history = model.fit(X_tr, np.array(y_tr), batch_size=32, epochs=5, validation_split=0.1, verbose=1)


Instructions for updating:
Use tf.cast instead.
Train on 6936 samples, validate on 771 samples
Epoch 1/5
6936/6936 [==============================] - 98s 14ms/step - loss: 0.3550 - acc: 0.9352 - f1_score: nan - val_loss: 0.2316 - val_acc: 0.9429 - val_f1_score: 0.9508
Epoch 2/5
6936/6936 [==============================] - 93s 13ms/step - loss: 0.2107 - acc: 0.9440 - f1_score: 0.9532 - val_loss: 0.1869 - val_acc: 0.9498 - val_f1_score: 0.9568
Epoch 3/5
6936/6936 [==============================] - 94s 14ms/step - loss: 0.1702 - acc: 0.9530 - f1_score: 0.9587 - val_loss: 0.1603 - val_acc: 0.9580 - val_f1_score: 0.9604
Epoch 4/5
6936/6936 [==============================] - 95s 14ms/step - loss: 0.1425 - acc: 0.9605 - f1_score: 0.9641 - val_loss: 0.1413 - val_acc: 0.9624 - val_f1_score: 0.9640
Epoch 5/5
6936/6936 [==============================] - 94s 14ms/step - loss: 0.1242 - acc: 0.9652 - f1_score: 0.9673 - val_loss: 0.1298 - val_acc: 0.9651 - val_f1_score: 0.9661


In [0]:
i = 60
p = model.predict(np.array([X_te[i]]))
p = np.argmax(p, axis=-1)
print("{:15} ({:5}): {}".format("Word", "True", "Pred"))
for w, pred in zip(X_te[i], p[0]):
    print("{:15}: {}".format(words[w], tags[pred]))

Word            (True ): Pred
In             : o
1978           : B-DT
,              : o
Raj            : o
Kapoor         : B-PER
direct         : o
Satyam         : o
Shivam         : o
Sundaram       : o
Lata           : B-PER
Mangeshkar     : I-PER
lends          : I-PER
her            : o
voice          : o
to             : o
the            : o
main           : o
theme          : o
song           : o
"              : o
Satyam         : o
Shivam         : o
Sundaram       : o
"              : o
,              : o
which          : o
was            : o
among          : o
the            : o
chart-toppers  : o
of             : o
the            : o
year           : o
.              : o
.              : o
.              : o
.              : o
.              : o
.              : o
.              : o
.              : o
.              : o
.              : o
.              : o
.              : o
.              : o
.              : o
.              : o
.              : o
.              : o


In [0]:
!pip install seqeval

In [0]:
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report


In [0]:
test_pred = model.predict(X_te, verbose=1)


857/857 [==============================] - 1s 1ms/step


In [0]:
idx2tag = {i: w for w, i in tag2idx.items()}

def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            p_i = np.argmax(p)
            out_i.append(idx2tag[p_i].replace("PAD", "O"))
        out.append(out_i)
    return out
    
pred_labels = pred2label(test_pred)
test_labels = pred2label(y_te)

In [0]:
print("F1-score: {:.1%}".format(f1_score(test_labels, pred_labels)))


F1-score: 55.9%


In [0]:
print(classification_report(test_labels, pred_labels))


             precision    recall  f1-score   support

          o       0.63      0.56      0.59      2300
        NUM       0.43      0.32      0.37       253
        EVE       0.00      0.00      0.00        16
        PER       0.69      0.62      0.65       457
         DT       0.66      0.63      0.65       267
        ORG       0.16      0.06      0.09       179
        LOC       0.52      0.64      0.58       349
        GPE       0.17      0.02      0.03       129
        ART       0.00      0.00      0.00        18

avg / total       0.58      0.52      0.54      3968



In [0]:
test_sentence = ["Rajath", "was", "here", "yesterday"]

In [0]:
x_test_sent = pad_sequences(sequences=[[word2idx.get(w, 0) for w in test_sentence]],
                            padding="post", value=0, maxlen=max_len)

In [0]:
p = model.predict(np.array([x_test_sent[0]]))
p = np.argmax(p, axis=-1)
print("{:15}||{}".format("Word", "Prediction"))
print(30 * "=")
for w, pred in zip(test_sentence, p[0]):
    print("{:15}: {:5}".format(w, tags[pred]))

Word           ||Prediction
Rajath         : o    
was            : o    
here           : o    
yesterday      : o    
